In [2]:
!pip install transformers torch nltk rouge_score

     |████████████████████████████████| 133 kB 7.4 MB/s eta 0:00:01
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=870add813cb0538fa9a34e9bff4e73d857b99b3534c1d71ed21c9e005690296a
  Stored in directory: /home/ubuntu/.cache/pip/wheels/24/55/6f/ebfc4cb176d1c9665da4e306e1705496206d08215c1acd9dde
Successfully built rouge-score


In [1]:
import math
import random
import sys
import os
# from framework.utilities.models import inference    
from framework.utilities.metrics import calculate_bleu, calculate_rouge
import fastText
from contextlib import contextmanager
import time
import json
import wandb
import torch.nn.functional as F
import torch
from collections import deque

In [2]:
from power_monitoring.monitor import HWMonitor
import threading
import wandb

In [3]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
# combined_outputs["gsm8k"] = [{"input_text": None, "7b": None, "tiny": None, "13b": None} for _ in range(3000)]
# len(combined_outputs["gsm8k"])

# with open('data/input_output_train', 'wb') as f:
#     pickle.dump(combined_outputs, f)

In [5]:
import pickle
import pandas as pd

with open('data/input_output_train', 'rb') as file:
    combined_outputs = pickle.load(file)

df_combined_outputs = pd.DataFrame(combined_outputs)
df_combined_outputs.head()

,wmt14,cnn_dailymail,gsm8k
0,{'input_text': 'Gutach: Noch mehr Sicherheit f...,{'input_text': '(CNN)The Palestinian Authority...,"{'input_text': None, '7b': None, 'tiny': None,..."
1,{'input_text': 'Sie stehen keine 100 Meter von...,{'input_text': '(CNN)Never mind cats having ni...,"{'input_text': None, '7b': None, 'tiny': None,..."
2,{'input_text': 'Zwei Anlagen so nah beieinande...,{'input_text': '(CNN)If you've been following ...,"{'input_text': None, '7b': None, 'tiny': None,..."
3,{'input_text': 'Diese Frage hat Gutachs Bürger...,{'input_text': '(CNN)Five Americans who were m...,"{'input_text': None, '7b': None, 'tiny': None,..."
4,"{'input_text': '""Die Rathausampel ist damals i...",{'input_text': '(CNN)A Duke student has admitt...,"{'input_text': None, '7b': None, 'tiny': None,..."


In [6]:
df_combined_outputs.size

9000

In [7]:
df_combined_outputs["cnn_dailymail"][3]

{'input_text': "(CNN)Five Americans who were monitored for three weeks at an Omaha, Nebraska, hospital after being exposed to Ebola in West Africa have been released, a Nebraska Medicine spokesman said in an email Wednesday. One of the five had a heart-related issue on Saturday and has been discharged but hasn't left the area, Taylor Wilson wrote. The others have already gone home. They were exposed to Ebola in Sierra Leone in March, but none developed the deadly virus. They are clinicians for Partners in Health, a Boston-based aid group. They all had contact with a colleague who was diagnosed with the disease and is being treated at the National Institutes of Health in Bethesda, Maryland. As of Monday, that health care worker is in fair condition. The Centers for Disease Control and Prevention in Atlanta has said the last of 17 patients who were being monitored are expected to be released by Thursday. More than 10,000 people have died in a West African epidemic of Ebola that dates to 

In [8]:
def algorithm(T, V, h_tilde, c):
    l_predictor, s_predictor = None, None
    Q = 0.0 
    results = []

    l_loss_window = deque(maxlen=100)  
    s_loss_window = deque(maxlen=100)
    acc_window = deque(maxlen=100)

    for t in range(1, T+1):
        print(f"Index: {t}")
        t_i = df_combined_outputs["wmt14"][t]["input_text"]
        t_e = df_combined_outputs["wmt14"][t]["label"]

        p_t = min(1, c / math.sqrt(t))
        X_t = Bernoulli(p_t)

        l_loss, s_loss = None, None

        g_large = 2.0
        g_small = 1.0  

        if X_t == 1:
            t_c = getResults(t, t_i, t_e)
            l_actual_acc = t_c['large_model_accuracy']
            s_actual_acc = t_c['small_model_accuracy']

            l_predictor, s_predictor, l_loss, s_loss = sgdStep(l_predictor, s_predictor, t_c)
            l_predictor, s_predictor = checkpoint(l_predictor, s_predictor, t, p_t)

            cost_large = V * g_large + Q * (h_tilde - l_actual_acc)
            cost_small = V * g_small + Q * (h_tilde - s_actual_acc)
        else:
            l_predicted_acc, s_predicted_acc = predict(t_i, l_predictor, s_predictor)

            cost_large = V * g_large + Q * (h_tilde - l_predicted_acc)
            cost_small = V * g_small + Q * (h_tilde - s_predicted_acc)

        if cost_large < cost_small:
            x_t = 'large'
            selected_acc = l_actual_acc if X_t == 1 else l_predicted_acc
        else:
            x_t = 'small'
            selected_acc = s_actual_acc if X_t == 1 else s_predicted_acc

        output, t_acc = querySelectedModel(t, t_e, x_t)

        if l_loss is not None:
            l_loss_window.append(l_loss)
        if s_loss is not None:
            s_loss_window.append(s_loss)
        acc_window.append(t_acc)

        smoothed_l_loss = moving_average(l_loss_window, window_size=1000)
        smoothed_s_loss = moving_average(s_loss_window, window_size=1000)
        smoothed_acc = moving_average(acc_window, window_size=1000)

        Q = max(0.0, Q + h_tilde - t_acc)
        log(results, t, t_i, output, smoothed_acc, smoothed_l_loss, smoothed_s_loss)

    return results

In [9]:
def log(results, t, t_i, output, t_acc, l_loss, s_loss):
    results.append({
        'iteration': t,
        'input': t_i,
        'best_output': output,
        'chosen_model_accuracy': t_acc,
        'l_loss': l_loss,
        's_loss': s_loss
    })
    log_data = {
        'iteration': t,
        'chosen_model_accuracy': t_acc
    }
        
    if l_loss is not None: log_data['l_loss'] = l_loss
    if s_loss is not None: log_data['s_loss'] = s_loss
        
    wandb.log(log_data)

In [10]:
def Bernoulli(p_t):
    return random.random() < p_t 

In [11]:
def querySelectedModel(t, t_e, x_t):
    if x_t == "large":
        output = df_combined_outputs["wmt14"][t]["13b"]
    else:
        output =  df_combined_outputs["wmt14"][t]["tiny"]
        
    return output, calculate_bleu(output, t_e)

In [12]:
# def queryBest(t, t_i, t_e, l_predictor, s_predictor):
#     l_acc, s_acc = predict(t_i, l_predictor, s_predictor)
#     if l_acc > s_acc: # use larger model, e.g. 7b llama
#         output = df_combined_outputs["wmt14"][t]["13b"]
#     else: # use smaller model for all other cases, e.g. 3b llama
#         output =  df_combined_outputs["wmt14"][t]["tiny"]

#     return output, calculate_bleu(output, t_e)
#     # return output, calculate_rouge(output, t_e)

In [13]:
def moving_average(data, window_size):
    data_list = list(data)  
    if len(data_list) == 0:
        return 0.0 
    if len(data_list) < window_size:
        return sum(data_list) / len(data_list)
    return sum(data_list[-window_size:]) / window_size

In [14]:
def compute_mse_loss(predicted_accuracy, true_accuracy):
    predicted_tensor = torch.tensor([predicted_accuracy], dtype=torch.float32)
    true_tensor = torch.tensor([true_accuracy], dtype=torch.float32)

    loss = F.mse_loss(predicted_tensor, true_tensor)
    
    return loss.item()

In [15]:
def sgdStep(large_model_predictor, small_model_predictor, t_c):
    predicted_large_label, predicted_small_label = 0, 0
    if large_model_predictor and small_model_predictor != None: 
        predicted_large_label = large_model_predictor.predict(t_c['input_text'])[0][0].replace("__label__", "")
        predicted_small_label = small_model_predictor.predict(t_c['input_text'])[0][0].replace("__label__", "")
    
    with open("fasttext_large.txt", "w") as f:
        f.write(f"__label__{t_c['large_model_accuracy']} {t_c['input_text']}\n")
    with open("fasttext_small.txt", "w") as f:
        f.write(f"__label__{t_c['small_model_accuracy']} {t_c['input_text']}\n")

    if os.path.exists("large_predictor.bin"):
        large_model_predictor = fastText.train_supervised(input="fasttext_large.txt", epoch=1, lr=1.0, wordNgrams=2, inputModel="large_predictor.bin")
    else:
        large_model_predictor = fastText.train_supervised(input="fasttext_large.txt", epoch=1, lr=1.0, wordNgrams=2)

    if os.path.exists("small_predictor.bin"):
        small_model_predictor = fastText.train_supervised(input="fasttext_small.txt", epoch=1, lr=1.0, wordNgrams=2, inputModel="small_predictor.bin")
    else:
        small_model_predictor = fastText.train_supervised(input="fasttext_small.txt", epoch=1, lr=1.0, wordNgrams=2)

    large_model_loss = compute_mse_loss(float(predicted_large_label), t_c['large_model_accuracy'])
    small_model_loss = compute_mse_loss(float(predicted_small_label), t_c['small_model_accuracy'])

    # print(f"Large Stats(predicted, actual): {predicted_large_label}, {t_c['large_model_accuracy']}")
    # print(f"Mean Square Error: {large_model_loss}")
    # print(f"Small Stats(predicted, actual): {predicted_small_label}, {t_c['small_model_accuracy']}")
    # print(f"Mean Square Error: {large_model_loss}")

    return large_model_predictor, small_model_predictor, large_model_loss, small_model_loss

In [16]:
def predict(text, l_predictor, s_predictor):
    l_predicted_label = l_predictor.predict(text)[0][0]
    l_predicted_accuracy = int(round(float(l_predicted_label.replace('__label__', ''))))

    s_predicted_label = s_predictor.predict(text)[0][0]
    s_predicted_accuracy = int(round(float(s_predicted_label.replace('__label__', ''))))

    return l_predicted_accuracy, s_predicted_accuracy

In [17]:
def getResults(t, t_i, t_e):
    s_output = df_combined_outputs["wmt14"][t]["tiny"]
    l_output = df_combined_outputs["wmt14"][t]["13b"]
    
    # WMT14
    s_acc = calculate_bleu(s_output, t_e)
    l_acc = calculate_bleu(l_output, t_e)

    # CNN Dailymail
    # s_acc = calculate_rouge(s_output, t_e)['rouge1']
    # l_acc = calculate_rouge(l_output, t_e)['rouge1']

    return {
        'input_text': t_i,
        'large_model_accuracy': l_acc,
        'small_model_accuracy': s_acc
    }

In [18]:
def checkpoint(l_predictor, s_predictor, t, p_t):
    if p_t > 0.5:
        if t % int(math.sqrt(t)) == 0:
            print("Checkpoint")
            l_predictor, s_predictor = save_models(l_predictor, s_predictor)
    else:
        if t % max(1, int(1 / math.sqrt(t))) == 0:
            print("Checkpoint")
            l_predictor, s_predictor = save_models(l_predictor, s_predictor)

    return l_predictor, s_predictor

In [19]:
def save_models(l_predictor, s_predictor):
    if l_predictor:
        l_predictor.save_model("large_predictor.bin")
    if s_predictor:
        s_predictor.save_model("small_predictor.bin")

    return l_predictor, s_predictor

In [20]:
def run_and_record(T, V, h_tilde, c):
    wandb.init(project="classifier")
    stop_event = threading.Event()
    hw_monitor = HWMonitor(monitoring_freq=1.0, stop_event=threading.Event())
    hw_monitor.start()

    
    results = algorithm(T, V, h_tilde, c)

    stop_event.set() 
    hw_monitor.join()
    wandb.finish()

    return results

In [21]:
results = run_and_record(2999, 100, .25, 5)

with open('experiments/fasttext/wmt14_results_5.json', 'w') as f:
    json.dump(results, f, indent=4)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ryanzhangofficial (ryzhangofficial). Use `wandb login --relogin` to force relogin


Index: 1


Read 0M words
Number of words:  27
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   19923 lr:  0.000000 loss:  0.000000 ETA:   0h 0m
Read 0M words
Number of words:  27
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   48437 lr:  0.000000 loss:  0.000000 ETA:   0h 0m


Checkpoint
Index: 2


Read 0M words
Number of words:  9
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   28863 lr:  0.000000 loss:  0.000000 ETA:   0h 0m
Read 0M words
Number of words:  9
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   45100 lr:  0.000000 loss:  0.000000 ETA:   0h 0m


...

Checkpoint
Index: 2970
Index: 2971
Index: 2972
Index: 2973
Index: 2974
Index: 2975
Index: 2976
Index: 2977
Index: 2978
Index: 2979
Index: 2980
Index: 2981
Index: 2982
Index: 2983
Index: 2984
Index: 2985
Index: 2986
Index: 2987
Index: 2988
Index: 2989
Index: 2990
Index: 2991
Index: 2992
Index: 2993
Index: 2994
Index: 2995
Index: 2996
Index: 2997
Index: 2998


Read 0M words
Number of words:  22
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   96938 lr:  0.000000 loss:  0.000000 ETA:   0h 0m
Read 0M words
Number of words:  22
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:  143125 lr:  0.000000 loss:  0.000000 ETA:   0h 0m0m


Checkpoint
Index: 2999


Exception in thread Thread-7:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/ubuntu/zry/Inferencing-Project/power_monitoring/monitor.py", line 43, in run
    wandb.log(all_res)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/wandb/sdk/wandb_run.py", line 452, in wrapper
    return func(self, *args, **kwargs)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/wandb/sdk/wandb_run.py", line 413, in wrapper_fn
    raise errors.UsageError(resolved_message)
wandb.errors.UsageError: Run (hfisrix3) is finished. The call to `log` will be ignored. Please make sure that you are using an active run.


bandwidth/disk_read_sys_bandwidth_mbs,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
bandwidth/disk_write_sys_bandwidth_mbs,▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁█▁▁▁▁▁▁▁▁██▁▁▁▁▁▁▁▁▁▁▁▁▁
bandwidth/net_recv_sys_bandwidth_mbs,▁▆▆▆▄▂▁▄▄█▁▁▆▆▁▂▄▂▂▄▄▁▅▂▄▅▁▆▄▄▂▁▁▄▄▄▅▁▅▁
bandwidth/net_sent_sys_bandwidth_mbs,▂▂▁▂▃▁▂▁▁▁▁▂▁▁▂▁▂▁▁▁█▂▁▂▁▂▁▂▃▁▁▁▂▁▁▁▁▁▂▂
chosen_model_accuracy,▂▃▃▃▃▃▅▁▂▂▃▃▂▄▄▃▃▅▄█▆▆▅▄▃▇▅▆▆▃▇▄▄▃▄▅▆▆▇▂
cpu/interrupts/global_ctx_switches_count,▁▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇█
cpu/interrupts/global_interrupts_count,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇█
cpu/interrupts/global_soft_interrupts_count,▁▁▁▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇▇▇▇█████
cpu/load/avg_sys_load_fifteen_min_percent,█████▇▇▇▇▇▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
cpu/load/avg_sys_load_five_min_percent,██▇▇▇▅▅▄▄▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
cpu/load/avg_sys_load_one_min_percent,█▅▄▄▄▅▅▄▅▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▁▁▁▁▁▁▁▁▁▁


In [22]:
results = run_and_record(2999, 100, .3, 5)

with open('experiments/fasttext/wmt14_results_3.json', 'w') as f:
    json.dump(results, f, indent=4)

Index: 1


Read 0M words
Number of words:  27
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   44477 lr:  0.000000 loss:  0.000000 ETA:   0h 0m
Read 0M words
Number of words:  27
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   86092 lr:  0.000000 loss:  0.000000 ETA:   0h 0m


Checkpoint
Index: 2


Read 0M words
Number of words:  9
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   51194 lr:  0.000000 loss:  0.000000 ETA:   0h 0mm
Read 0M words
Number of words:  9
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   46720 lr:  0.000000 loss:  0.000000 ETA:   0h 0m


Checkpoint
Index: 3


...

Checkpoint
Index: 2977
Index: 2978
Index: 2979
Index: 2980
Index: 2981
Index: 2982
Index: 2983


Read 0M words
Number of words:  9
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   15587 lr:  0.000000 loss:  0.000000 ETA:   0h 0m0m
Read 0M words
Number of words:  9
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   70452 lr:  0.000000 loss:  0.000000 ETA:   0h 0m


Checkpoint
Index: 2984
Index: 2985
Index: 2986
Index: 2987
Index: 2988
Index: 2989
Index: 2990
Index: 2991
Index: 2992
Index: 2993
Index: 2994
Index: 2995
Index: 2996
Index: 2997
Index: 2998
Index: 2999


Exception in thread Thread-18:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/ubuntu/zry/Inferencing-Project/power_monitoring/monitor.py", line 43, in run
    wandb.log(all_res)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/wandb/sdk/wandb_run.py", line 452, in wrapper
    return func(self, *args, **kwargs)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/wandb/sdk/wandb_run.py", line 413, in wrapper_fn
    raise errors.UsageError(resolved_message)
wandb.errors.UsageError: Run (c45jtjyu) is finished. The call to `log` will be ignored. Please make sure that you are using an active run.


bandwidth/disk_read_sys_bandwidth_mbs,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
bandwidth/disk_write_sys_bandwidth_mbs,▁▁▁█▁▁▁▁▇▇▇▁▇▁▇▇▇▁▁▁█▇▇▁▁▁▁▁▁▁▁▁▇▁▁▁▁▁▁▁
bandwidth/net_recv_sys_bandwidth_mbs,▁▁▁▆▁▅▁▂▁▄▁▁▆▅▁▁▅▄▁▁▁▄▁▁▃▁▄▆▁▁▄▁▁▅▁▃▄▅█▂
bandwidth/net_sent_sys_bandwidth_mbs,▁▁█▂▁▁▁▂▁▁▂▁▂▁▂▁▂▂▁▁▁▁▂▂▃▁▂▂▁▂▁▁▁▁▁▃▂▁▁▁
chosen_model_accuracy,▁▂▃▄▅▁▃▃▃▃▄▄▃█▅▆▇▃▄▆▅▅▅▅▆▅▂▁▂▃▄▄▅▅▅▅▅▄▄▃
cpu/interrupts/global_ctx_switches_count,▁▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇████
cpu/interrupts/global_interrupts_count,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇██
cpu/interrupts/global_soft_interrupts_count,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
cpu/load/avg_sys_load_fifteen_min_percent,▄▅▅▅▅▄▅▄▄▆▅█▇▆▆▅▄▄▃▂▃▄▄▃▂▁▁▁▁▁▃▃▄▄▄▃▃▂▂▂
cpu/load/avg_sys_load_five_min_percent,▁▄▃▂▂▄▃▃▅▄██▆▅▅▅▅▄▄▂▁▂▃▅▂▁▂▁▂▂▆▆▅▂▂▂▂▂▂▄
cpu/load/avg_sys_load_one_min_percent,▃▃▂▂▂▁▁▄▃▃▄▄▂█▃▃▂▂▂▁▁▂▄▃▁▁▁▃▂▂▃▃▂▂▃▂▂▂▁▂


In [28]:
results = run_and_record(2999, 100, .35, 5)

with open('experiments/fasttext/V100-fasttext-wmt14-h35', 'w') as f:
    json.dump(results, f, indent=4)

Index: 1


Read 0M words
Number of words:  27
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   92391 lr:  0.000000 loss:  0.000000 ETA:   0h 0m
Read 0M words
Number of words:  27
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:  146422 lr:  0.000000 loss:  0.000000 ETA:   0h 0m


Checkpoint
Index: 2


Read 0M words
Number of words:  9
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   14448 lr:  0.000000 loss:  0.000000 ETA:   0h 0m0m
Read 0M words
Number of words:  9
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   35205 lr:  0.000000 loss:  0.000000 ETA:   0h 0m


Checkpoint
Index: 3


...

Checkpoint
Index: 2991
Index: 2992
Index: 2993
Index: 2994
Index: 2995
Index: 2996
Index: 2997
Index: 2998


Read 0M words
Number of words:  22
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:  183427 lr:  0.000000 loss:  0.000000 ETA:   0h 0m
Read 0M words
Number of words:  22
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:  131316 lr:  0.000000 loss:  0.000000 ETA:   0h 0m


Checkpoint
Index: 2999


Exception in thread Thread-58:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/ubuntu/zry/Inferencing-Project/power_monitoring/monitor.py", line 43, in run
    wandb.log(all_res)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/wandb/sdk/wandb_run.py", line 452, in wrapper
    return func(self, *args, **kwargs)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/wandb/sdk/wandb_run.py", line 413, in wrapper_fn
    raise errors.UsageError(resolved_message)
wandb.errors.UsageError: Run (b319ea1l) is finished. The call to `log` will be ignored. Please make sure that you are using an active run.


bandwidth/disk_read_sys_bandwidth_mbs,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
bandwidth/disk_write_sys_bandwidth_mbs,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇█▁▁█▁▁██▁▁▁▁▁█▇█▁▁▁▁▁▁▁
bandwidth/net_recv_sys_bandwidth_mbs,▂▂▃▃▃▃▄▃▂▂▂▂▂▄▃▂▁▁▂▄█▂▂▆▂▃▁▂▄▃▂▂▁▂▂▂▁▅▂▄
bandwidth/net_sent_sys_bandwidth_mbs,▁▁▁▁▃▁▁▁▂▁▁▂▁▂▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▅▄▁
chosen_model_accuracy,▁▂▃▁▁▂▂▃▄▂▃▃▄▂▇███▅▆▅▄▃▅▃▃▂▃▃▃▄▅▄▄▃▇▂▄▄▅
cpu/interrupts/global_ctx_switches_count,▁▁▁▁▁▁▁▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇███
cpu/interrupts/global_interrupts_count,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇█
cpu/interrupts/global_soft_interrupts_count,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▇▇▇▇███
cpu/load/avg_sys_load_fifteen_min_percent,▁▁▂▂▃▃▄▄▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████
cpu/load/avg_sys_load_five_min_percent,▁▁▃▄▅▆▆▆▆▇██▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████▇▇▇▇▇▇▇
cpu/load/avg_sys_load_one_min_percent,▁▄▅▅▆▆▆▆▆▆▆▆▆▆▆▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆██▆▆▆▆▆▆▆▆


In [29]:
results = run_and_record(2999, 500, .25, 5)

with open('experiments/fasttext/V500-fasttext-wmt14-h25', 'w') as f:
    json.dump(results, f, indent=4)

Index: 1


Read 0M words
Number of words:  27
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   72832 lr:  0.000000 loss:  0.000000 ETA:   0h 0m
Read 0M words
Number of words:  27
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   69850 lr:  0.000000 loss:  0.000000 ETA:   0h 0m


Checkpoint
Index: 2


Read 0M words
Number of words:  9
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   23384 lr:  0.000000 loss:  0.000000 ETA:   0h 0m0m
Read 0M words
Number of words:  9
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   45022 lr:  0.000000 loss:  0.000000 ETA:   0h 0m


Checkpoint
Index: 3


...

Checkpoint
Index: 2986


Read 0M words
Number of words:  12
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   43833 lr:  0.000000 loss:  0.000000 ETA:   0h 0m
Read 0M words
Number of words:  12
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   71053 lr:  0.000000 loss:  0.000000 ETA:   0h 0m


Checkpoint
Index: 2987
Index: 2988
Index: 2989
Index: 2990
Index: 2991
Index: 2992
Index: 2993
Index: 2994
Index: 2995
Index: 2996
Index: 2997
Index: 2998
Index: 2999


Exception in thread Thread-68:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/ubuntu/zry/Inferencing-Project/power_monitoring/monitor.py", line 43, in run
    wandb.log(all_res)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/wandb/sdk/wandb_run.py", line 452, in wrapper
    return func(self, *args, **kwargs)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/wandb/sdk/wandb_run.py", line 413, in wrapper_fn
    raise errors.UsageError(resolved_message)
wandb.errors.UsageError: Run (9swyiw0a) is finished. The call to `log` will be ignored. Please make sure that you are using an active run.


bandwidth/disk_read_sys_bandwidth_mbs,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
bandwidth/disk_write_sys_bandwidth_mbs,█▁▁▁▁▁▁▁▁▁▁▁█▁█▁▁█▁▁▁▁██▁▁▁▁▁█▁▁█▁▁▁▁▁█▁
bandwidth/net_recv_sys_bandwidth_mbs,▁▆▆▂▂▁█▅▂▆▄▄▁▁▃▄▆▁▄▁▂▃▆▆▂▁▄▁▁▅▁▅▂▆▁▂▇▁▃▃
bandwidth/net_sent_sys_bandwidth_mbs,▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁
chosen_model_accuracy,▁▃▃▃▆▆▅▄▃▄▄▄▄▄▄▅▄▆█▆▆▇▄▄▅▅▅▆▄▄▅▄▃▇▆▅▅▅▇▃
cpu/interrupts/global_ctx_switches_count,▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▆▆▇███
cpu/interrupts/global_interrupts_count,▁▁▁▁▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇████
cpu/interrupts/global_soft_interrupts_count,▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇█
cpu/load/avg_sys_load_fifteen_min_percent,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▅▅▅▆▆▆▇▇▇▇▇▇▇███████████
cpu/load/avg_sys_load_five_min_percent,▁▁▁▁▁▁▁▁▂▂▁▁▁▂▂▄▅▅▆▆▇▇▇▇▇██████████▇▇▇▇▇
cpu/load/avg_sys_load_one_min_percent,▂▂▂▂▂▁▄▃▂▁▂▁▁▁▁▁▁▅▅▆▆▇▇▇▇▇▇▇▇▇▇██▇▇▇▇▇▇▇


In [30]:
results = run_and_record(2999, 1000, .25, 5)

with open('experiments/fasttext/V1000-fasttext-wmt14-h25', 'w') as f:
    json.dump(results, f, indent=4)

Index: 1


Read 0M words
Number of words:  27
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   28832 lr:  0.000000 loss:  0.000000 ETA:   0h 0m
Read 0M words
Number of words:  27
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   85243 lr:  0.000000 loss:  0.000000 ETA:   0h 0m


Checkpoint
Index: 2


Read 0M words
Number of words:  9
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   21966 lr:  0.000000 loss:  0.000000 ETA:   0h 0mm
Read 0M words
Number of words:  9
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   31466 lr:  0.000000 loss:  0.000000 ETA:   0h 0m


Checkpoint
Index: 3


...

Checkpoint
Index: 2999


Read 0M words
Number of words:  18
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   38501 lr:  0.000000 loss:  0.000000 ETA:   0h 0m
Read 0M words
Number of words:  18
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   78504 lr:  0.000000 loss:  0.000000 ETA:   0h 0m


Checkpoint


Exception in thread Thread-78:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/ubuntu/zry/Inferencing-Project/power_monitoring/monitor.py", line 43, in run
    wandb.log(all_res)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/wandb/sdk/wandb_run.py", line 452, in wrapper
    return func(self, *args, **kwargs)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/wandb/sdk/wandb_run.py", line 413, in wrapper_fn
    raise errors.UsageError(resolved_message)
wandb.errors.UsageError: Run (69ujbi6q) is finished. The call to `log` will be ignored. Please make sure that you are using an active run.


bandwidth/disk_read_sys_bandwidth_mbs,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
bandwidth/disk_write_sys_bandwidth_mbs,▁▁▁▁█▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇█▁▁▁▁▁█▁▁▁▁▁▁
bandwidth/net_recv_sys_bandwidth_mbs,▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁
bandwidth/net_sent_sys_bandwidth_mbs,▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁
chosen_model_accuracy,▁▃▅▄▅▁▂▂▂▃▃▅▄▁▅▃▄██▆▆▄▃▃▄▃▂▃▅▅▄▄▆▆▄▄▆▆▃▂
cpu/interrupts/global_ctx_switches_count,▁▁▁▁▁▁▂▂▂▂▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█
cpu/interrupts/global_interrupts_count,▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇██
cpu/interrupts/global_soft_interrupts_count,▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇█
cpu/load/avg_sys_load_fifteen_min_percent,▇▇▇▇▇███████▇▇▇▆▄▄▄▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
cpu/load/avg_sys_load_five_min_percent,▇▇▇▇▇███████████▆▅▄▄▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁
cpu/load/avg_sys_load_one_min_percent,▇▇▇▇█▇▇▇█▇▇▇▇▇▄▂▂▂▂▂▁▂▂▂▂▂▂▂▁▁▁▁▁▁▂▁▃▂▂▃


In [ ]:
results = run_and_record(2999, 10000, .25, 5)

with open('experiments/fasttext/V10000-fasttext-wmt14-h25', 'w') as f:
    json.dump(results, f, indent=4)

Index: 1


Read 0M words
Number of words:  27
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   23654 lr:  0.000000 loss:  0.000000 ETA:   0h 0m
Read 0M words
Number of words:  27
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   89483 lr:  0.000000 loss:  0.000000 ETA:   0h 0m


Checkpoint
Index: 2


Read 0M words
Number of words:  9
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   23359 lr:  0.000000 loss:  0.000000 ETA:   0h 0m0m
Read 0M words
Number of words:  9
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   25219 lr:  0.000000 loss:  0.000000 ETA:   0h 0m


Checkpoint
Index: 3


...

Read 0M words
Number of words:  15
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   89136 lr:  0.000000 loss:  0.000000 ETA:   0h 0m0m
Read 0M words
Number of words:  15
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   20099 lr:  0.000000 loss:  0.000000 ETA:   0h 0m


Checkpoint
Index: 2197
Index: 2198
Index: 2199
Index: 2200
Index: 2201
Index: 2202
Index: 2203
Index: 2204
Index: 2205
Index: 2206


Read 0M words
Number of words:  12
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   96969 lr:  0.000000 loss:  0.000000 ETA:   0h 0m
Read 0M words
Number of words:  12
Number of labels: 1
start training...
Progress: 100.0% words/sec/thread:   20816 lr:  0.000000 loss:  0.000000 ETA:   0h 0m


Checkpoint
Index: 2207
Index: 2208
Index: 2209
Index: 2210
Index: 2211
Index: 2212
Index: 2213
Index: 2214
Index: 2215
Index: 2216
Index: 2217


Read 0M words
Number of words:  13
Number of labels: 1


In [26]:
import json

with open('experiments/fasttext/wmt14_results_1.json', 'r') as f:
    data = json.load(f)

for result in data[:10]:
    print(result['chosen_model_accuracy'])

0.1059238520965933
0.07436771913785759
0.05671674131893009
0.07413055401887111
0.09897532612346267
0.08602798791948046
0.12735302338091695
0.11664138006661937
0.12813830630746953
0.1258403278005667


In [27]:
total = 0
num_results = len(data)

for result in data:
    total += result['chosen_model_accuracy']

total / num_results

0.18898695987501102

In [16]:
with open('data/cnndailymail_results.json', 'r') as f:
    data = json.load(f)

In [17]:
rouge1_total = 0
rouge2_total = 0
rougeL_total = 0

num_results = len(data)

for result in data:
    rouge1_total += result['chosen_model_accuracy']['rouge1']
    rouge2_total += result['chosen_model_accuracy']['rouge2']
    rougeL_total += result['chosen_model_accuracy']['rougeL']

rouge1_avg = rouge1_total / num_results
rouge2_avg = rouge2_total / num_results
rougeL_avg = rougeL_total / num_results

print(f"Average ROUGE-1: {rouge1_avg}")
print(f"Average ROUGE-2: {rouge2_avg}")
print(f"Average ROUGE-L: {rougeL_avg}")

Average ROUGE-1: 0.2937870737089562
Average ROUGE-2: 0.10789698165405123
Average ROUGE-L: 0.19841589354653624
